<a href="https://colab.research.google.com/github/Fr33zz/mw/blob/master/14%20get_new_table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import math
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
orders = gc.open_by_url('https://docs.google.com/spreadsheets/d/106Mk7_n9iXmCB-fuPFgpWsV1t7JtoAdPaePi5x8BFTM')

payments = gc.open_by_url('https://docs.google.com/spreadsheets/d/1OKkFUxN-a55K52Z6QO0wnZg9SB-_6d3VZ_BXDCYsGYc')

closed = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Gs7OkvQLNpt5IDTrsP5PBWbrSTO89zOPysbpbp7uzrQ')

sch_pay = gc.open_by_url('https://docs.google.com/spreadsheets/d/1AkH2q9AGbFuaXW3fUlrvvzcObFAr0vOkYdr2sNb4uLw')

opened = gc.open_by_url('https://docs.google.com/spreadsheets/d/1xUc4ayCcVNpkT1Bcx8_Hk84g02iM3U5GkxSSbZMMGAk')

charge = gc.open_by_url('https://docs.google.com/spreadsheets/d/1gOgXDjkxdjNaE-O-jprV77Tucdqm8uE-vg0NHDxP8xk')


def get_data(spreadsheet, list_number=0):
  ind = pd.DataFrame(spreadsheet.worksheets()[list_number].get_all_values())
  ind = ind.rename(columns=ind.iloc[0]).drop(index=0).reset_index().drop(columns='index')
  return ind

def get_titles(x, type_=1):
  '''type = {0:"list", 1:"df"}'''
  if type_==0:
    return [i.title for i in x.worksheets()]
  if type_==1:
    return pd.DataFrame([i.title for i in x.worksheets()])


def to_num(data, columns, print_=False):
  d = data.copy()
  for i in columns:
    try:       
      d[i] = d[i].map(lambda x: '.'.join(''.join(''.join(str(x).split('\xa0')).split(' ')).split(',')) if x!='' else np.nan).astype('float')
    except:
      if print_:
        print(i)
      else:
        None
  return d

#for i in dates:
#  d[i] = pd.to_datetime(d[i].replace({'':np.nan}), format='%d.%m.%Y')

In [ ]:
loan = get_data(opened, 0)
loan = loan[loan['Пауза']!='На паузе']
loan = loan[loan['']!='На паузе']
loan  = to_num( loan,  loan.iloc[:,-17:].columns, True)

Резерв по основному долгу, %
Резерв по процентам, %



In [ ]:
cols_loan = ['Дата', '№ договора', 'Дата попадания в реестр',
       'ФИО','Дата рождения',
       'Полученная сумма заемщиком', 'Ежемесячный взнос',
       'Сумма начисленных процентов',
       'Срок просрочки (в днях)', 'Категория просрочки','Общая сумма задолженности',
       'Общая сумма текущей задолженности', 'Общая сумма просроченной задолженности']

loan['Общая сумма текущей задолженности'] = loan[['Текущая задолженность по основному долгу',
                                              'Текущая задолженность по процентам',
                                              'Текущая задолженность по комиссии']].sum(axis=1)

loan['Общая сумма просроченной задолженности'] = loan['Общая сумма задолженности'] - loan['Общая сумма текущей задолженности']

loan = loan[cols_loan]

dates = ['Дата', 'Дата попадания в реестр', 'Дата рождения']
for i in dates:
  loan[i] = pd.to_datetime(loan[i].replace({'':np.nan}), format='%d.%m.%Y')

loan  = to_num( loan,  loan.iloc[:,-8:].columns, True)

Категория просрочки


In [ ]:
loan['score'] = loan['Общая сумма просроченной задолженности'] / loan['Срок просрочки (в днях)']

loan = loan[loan['Срок просрочки (в днях)']>0]

#plt.bar(range(loan.shape[0]), height = loan.sort_values('score', ascending=False)['Срок просрочки (в днях)'].values)
loan = loan.sort_values('score', ascending=False)
loan = loan.reset_index().reset_index().drop(columns=['level_0', 'index']).reset_index()

loan['index'] = loan['index']//100%2

In [ ]:
for i in loan[dates]:
  loan[i] = loan[i].map(lambda x: str(x).split(' ')[0])

In [ ]:
d = pd.read_csv('/content/drive/My Drive/analytics/14 callcenter_charge_strategy/mw.csv')
#d = d[['id', 'masked_id']]

loan = pd.merge(left=loan, right=d, left_on='№ договора', right_on='masked_id',how='left').iloc[:,:-1]

loan['id'] = loan['id'].fillna('').astype(str).map(lambda x: 'https://moneywall.io/admin/credit_agreements/'+x.split('.')[0])

In [ ]:
d

,id,state,masked_id,expiration_date,user_id,holder_name,active
0,4062,closed,MW333350425,10/22,198.0,MAXIM GORSHKOV,True
1,3435,closed,MW333356746,01/24,6288.0,KARAVAEVA IRINA,True
2,1088,closed,MW333331740,06/21,1522.0,Irina Poleshchuk,True
3,4308,closed,MW333351928,05/22,11082.0,ANNA ZATSEPINA,True
4,5217,closed,MW333367972,08/20,13274.0,VAGRAM MALKHASYAN,False
...,...,...,...,...,...,...,...
7874,8039,closed,MW333396503,04/23,19735.0,OLESYA GUK,True
7875,8204,open,MW333391563,05/23,17426.0,MARINA KONDIENKO,True
7876,8205,open,MW333391591,12/28,24699.0,PAVEL LAGUNOV,True
7877,8008,closed,MW333395038,01/24,11703.0,SERGEY OVCHARENKO,True


In [ ]:
loan['status'] = ''
loan['списанная сумма'] = ''
loan['Попытки'] = ''  #количество неудачных попыток
loan['коммент'] = ''
loan['год-месяц'] = loan['Дата попадания в реестр'].map(lambda x: '-'.join(x.split('-')[:2]))

cols = ['id', 
        'index', 
        '№ договора',
        'status',
        'списанная сумма',
        'Попытки',
        'коммент',
        #'score',
        'год-месяц',
        'Срок просрочки (в днях)',
        'Общая сумма просроченной задолженности',
        'Общая сумма текущей задолженности',
        'Общая сумма задолженности',
        'Полученная сумма заемщиком',
        'Ежемесячный взнос',
        'Сумма начисленных процентов',
        'Дата']

In [ ]:
loan

,index,Дата,№ договора,Дата попадания в реестр,ФИО,Дата рождения,Полученная сумма заемщиком,Ежемесячный взнос,Сумма начисленных процентов,Срок просрочки (в днях),Категория просрочки,Общая сумма задолженности,Общая сумма текущей задолженности,Общая сумма просроченной задолженности,score,id,state,masked_id,expiration_date,user_id,holder_name,status,списанная сумма,Попытки,коммент,год-месяц
0,0,2021-02-02,MW333399077,2021-02-02,Berdagulova K. V.,1988-10-24,7097.0,4797.0,1129.58,1.0,от 1 до 30,9594.00,4797.0,4797.00,4797.000000,https://moneywall.io/admin/credit_agreements/8158,open,MW333399077,05/23,19549.0,KARINA BERDAGULOVA,,,,,2021-02
1,0,2021-02-02,MW333398202,2021-01-29,Прокопенко М. П.,2000-08-16,36587.0,24731.0,6307.22,9.0,от 1 до 30,49891.92,24731.0,25160.92,2795.657778,https://moneywall.io/admin/credit_agreements/8137,open,MW333398202,11/23,23756.0,MARIA PROKOPENKO,,,,,2021-01
2,0,2021-02-02,MW333399905,2021-02-01,Казарцев Д. Г.,1987-07-17,6332.4,4279.0,1018.35,2.0,от 1 до 30,8571.47,4279.0,4292.47,2146.235000,https://moneywall.io/admin/credit_agreements/8156,open,MW333399905,09/23,24176.0,DMITRII KAZARTSEV,,,,,2021-02
3,0,2021-02-02,MW333399856,2021-01-30,Abduragimova A. -.,2001-01-16,7081.0,4785.0,1162.14,4.0,от 1 до 30,9617.10,4785.0,4832.10,1208.025000,https://moneywall.io/admin/credit_agreements/8150,open,MW333399856,09/22,23965.0,AIBIKE ABDURAGIMOVA,,,,,2021-01
4,0,2021-02-02,MW333396121,2021-02-01,Макарова Я. Ю.,1996-01-29,3374.8,2281.0,587.86,2.0,от 1 до 30,2289.16,0.0,2289.16,1144.580000,https://moneywall.io/admin/credit_agreements/8068,closed,MW333396121,11/23,14580.0,YANA MAKAROVA,,,,,2021-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2190,1,2021-02-02,MW333338881,2020-03-19,Серен-оол А. Е.,1991-04-04,5217.0,2849.0,481.00,932.0,от 361,824.00,0.0,824.00,0.884120,https://moneywall.io/admin/credit_agreements/688,open,MW333338881,06/19,2710.0,AYLANA SEREN-OOL,,,,,2020-03
2191,1,2021-02-02,R877111,2016-05-26,ВЫСОЦКИЙ М.Г.,NaT,1202.0,748.0,294.00,1714.0,от 361,1498.00,0.0,1498.00,0.873979,https://moneywall.io/admin/credit_agreements/,NaN,NaN,NaN,NaN,NaN,,,,,2016-05
2192,1,2021-02-02,R877098,2016-05-16,ВЫСОЦКИЙ М.Г.,NaT,1202.0,748.0,294.00,1724.0,от 361,1498.00,0.0,1498.00,0.868910,https://moneywall.io/admin/credit_agreements/,NaN,NaN,NaN,NaN,NaN,,,,,2016-05
2193,1,2021-02-02,R876868,2016-02-05,ДЫРДОВ В.А.,NaT,1349.0,801.0,253.00,1825.0,от 361,1551.00,0.0,1551.00,0.849863,https://moneywall.io/admin/credit_agreements/,NaN,NaN,NaN,NaN,NaN,,,,,2016-02


In [ ]:
loan[cols].to_excel('/content/drive/My Drive/analytics/14 callcenter_charge_strategy/charge_strategy_raw.xlsx')

In [ ]:
d

,id,state,masked_id,expiration_date,user_id,holder_name,active
0,4062,closed,MW333350425,10/22,198.0,MAXIM GORSHKOV,True
1,3435,closed,MW333356746,01/24,6288.0,KARAVAEVA IRINA,True
2,1088,closed,MW333331740,06/21,1522.0,Irina Poleshchuk,True
3,4308,closed,MW333351928,05/22,11082.0,ANNA ZATSEPINA,True
4,5217,closed,MW333367972,08/20,13274.0,VAGRAM MALKHASYAN,False
...,...,...,...,...,...,...,...
7874,8039,closed,MW333396503,04/23,19735.0,OLESYA GUK,True
7875,8204,open,MW333391563,05/23,17426.0,MARINA KONDIENKO,True
7876,8205,open,MW333391591,12/28,24699.0,PAVEL LAGUNOV,True
7877,8008,closed,MW333395038,01/24,11703.0,SERGEY OVCHARENKO,True


In [ ]:
drop2[0]

NameError: ignored

In [ ]:
d['active'].fillna(True)

id2save = d[d['active'].fillna(True)]['masked_id'].values

new_loan = loan[loan['masked_id'].isin(id2save)]

#d0 = get_data(charge, 0)
#d1 = get_data(charge, 1)

#not2drop = np.unique(np.concatenate((d0[d0['status']=='']['№ договора'].values,
#                                     d1[d1['status']=='']['№ договора'].values), axis=0))

drop2 = [get_data(charge, i) for i in range(2)]
drop2 = [i[i['status']!='']['№ договора'].values for i in drop2]
drop2 = np.concatenate(drop2, axis=0)

new_loan = new_loan[~new_loan['№ договора'].isin(drop2)]

In [ ]:
drop2 = [get_data(charge, i) for i in range(2)]
# drop2 = [i[i['status']!='']['№ договора'].values for i in drop2]
# drop2 = np.concatenate(drop2, axis=0)

In [ ]:
drop2[0]


,level_0,,id,№ договора,status,списанная сумма,Попытки,коммент,год-месяц,Срок просрочки (в днях),Общая сумма просроченной задолженности,Общая сумма текущей задолженности,Общая сумма задолженности,Полученная сумма заемщиком,Ежемесячный взнос,Сумма начисленных процентов,Дата
0,1,0,https://moneywall.io/admin/credit_agreements/8158,MW333399077,,,,,2021-02,1,4797,4797,9594,7097,4797,"1129,58",2021-02-02
1,2,2,https://moneywall.io/admin/credit_agreements/8156,MW333399905,,,,,2021-02,2,"4292,47",4279,"8571,47","6332,4",4279,"1018,35",2021-02-02
2,3,3,https://moneywall.io/admin/credit_agreements/8150,MW333399856,,,,,2021-01,4,"4832,1",4785,"9617,1",7081,4785,"1162,14",2021-02-02
3,4,4,https://moneywall.io/admin/credit_agreements/8068,MW333396121,,,,,2021-02,2,"2289,16",0,"2289,16","3374,8",2281,"587,86",2021-02-02
4,5,6,https://moneywall.io/admin/credit_agreements/8154,MW333399885,,,,,2021-01,4,"4077,61",4038,"8115,61",5974,4038,"980,45",2021-02-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902,903,2169,https://moneywall.io/admin/credit_agreements/848,MW333339982,,,,,2020-07,881,1297,0,1297,11631,6281,931,2021-02-02
903,904,2186,https://moneywall.io/admin/credit_agreements/1360,MW333343659,,,,,2020-09,683,697,0,697,3592,1932,272,2021-02-02
904,905,2187,https://moneywall.io/admin/credit_agreements/4758,MW333364748,,,,,2020-03,339,"342,81",0,"342,81","2003,4",1354,"68,23",2021-02-02
905,906,2189,https://moneywall.io/admin/credit_agreements/1505,MW333344049,,,,,2020-03,640,568,0,568,12469,6708,947,2021-02-02


In [ ]:
len(drop2)

0

In [ ]:
new_loan[new_loan['год-месяц'] == '2020-12']

,index,Дата,№ договора,Дата попадания в реестр,ФИО,Дата рождения,Полученная сумма заемщиком,Ежемесячный взнос,Сумма начисленных процентов,Срок просрочки (в днях),Категория просрочки,Общая сумма задолженности,Общая сумма текущей задолженности,Общая сумма просроченной задолженности,score,id,state,masked_id,expiration_date,user_id,holder_name,status,списанная сумма,Попытки,коммент,год-месяц
18,0,2021-02-02,MW333385234,2020-12-30,Alena Z. V.,1998-06-28,27282.00,18441.0,15851.24,109.0,от 91 до 120,52260.96,0.0,52260.96,479.458349,https://moneywall.io/admin/credit_agreements/7488,open,MW333385234,12/23,20008.0,ALENA ZHIVCHIKOVA,,,,,2020-12
19,0,2021-02-02,MW333361746,2020-12-26,Косых О. А.,1970-10-01,56085.00,37911.0,53078.24,273.0,от 271 до 360,128079.12,0.0,128079.12,469.154286,https://moneywall.io/admin/credit_agreements/5765,open,MW333361746,08/22,14536.0,OXANA KOSYKH,,,,,2020-12
33,0,2021-02-02,MW333386626,2020-12-11,Майорова Д. А.,1998-10-12,20286.00,13712.0,11575.30,106.0,от 91 до 120,38206.52,0.0,38206.52,360.438868,https://moneywall.io/admin/credit_agreements/7550,open,MW333386626,10/22,20167.0,DARIA MAIOROVA,,,,,2020-12
51,0,2021-02-02,MW333384006,2020-12-09,Sobolevskaia K. E.,1990-12-28,18563.00,12547.0,11366.52,118.0,от 91 до 120,34828.98,0.0,34828.98,295.160847,https://moneywall.io/admin/credit_agreements/7326,open,MW333384006,02/25,18043.0,KSENIYA SOBOLEVSKAYA,,,,,2020-12
93,0,2021-02-02,MW333358175,2020-12-30,Каратеев М. К.,1977-04-23,47424.00,32056.0,39600.49,415.0,от 361,89348.24,0.0,89348.24,215.296964,https://moneywall.io/admin/credit_agreements/3847,open,MW333358175,07/22,8554.0,MIKHAIL KARATEEV,,,,,2020-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1505,1,2021-02-02,R880045,2020-12-18,Рахмангулова Л. Г.,1996-08-28,3792.00,2144.0,288.00,650.0,от 361,6585.00,0.0,6585.00,10.130769,https://moneywall.io/admin/credit_agreements/2042,open,R880045,NaN,6062.0,RAKHMANGULOVA LIANA,,,,,2020-12
1645,0,2021-02-02,MW333377681,2020-12-26,Безуглая В. З.,1999-07-09,3401.00,2298.0,758.24,187.0,от 181 до 270,1456.22,0.0,1456.22,7.787273,https://moneywall.io/admin/credit_agreements/6389,open,MW333377681,01/23,14136.0,MOMENTUM R,,,,,2020-12
1818,0,2021-02-02,MW333342276,2020-12-27,Стручков К. К.,2000-01-09,11098.00,7502.0,9406.39,499.0,от 361,2649.60,0.0,2649.60,5.309820,https://moneywall.io/admin/credit_agreements/2845,open,MW333342276,04/22,6194.0,KONSTANTIN STRUCHKOV,,,,,2020-12
2007,0,2021-02-02,MW333369473,2020-12-15,Имамутдинов А. И.,2000-11-01,787.74,532.0,563.61,293.0,от 271 до 360,1008.13,0.0,1008.13,3.440717,https://moneywall.io/admin/credit_agreements/5414,open,MW333369473,01/23,13623.0,ADEL IMAMUTDINOV,,,,,2020-12


In [ ]:
drop2

In [ ]:
m = new_loan['score'].max()
new_loan['score'] = new_loan['score'] + (new_loan['Срок просрочки (в днях)']<6)*m

new_loan = new_loan.sort_values('score', ascending=False)

In [ ]:
new_loan[new_loan['id'].map(lambda x: x[-1]!='/')][cols]\
  .to_excel('/content/drive/My Drive/analytics/14 callcenter_charge_strategy/charge_strategy.xlsx')

In [ ]:
new_loan[cols]#.sort_values('№ договора')

,id,index,№ договора,status,списанная сумма,Попытки,коммент,год-месяц,Срок просрочки (в днях),Общая сумма просроченной задолженности,Общая сумма текущей задолженности,Общая сумма задолженности,Полученная сумма заемщиком,Ежемесячный взнос,Сумма начисленных процентов,Дата
0,https://moneywall.io/admin/credit_agreements/8158,0,MW333399077,,,,,2021-02,1.0,4797.00,4797.0,9594.00,7097.0,4797.0,1129.58,2021-02-02
2,https://moneywall.io/admin/credit_agreements/8156,0,MW333399905,,,,,2021-02,2.0,4292.47,4279.0,8571.47,6332.4,4279.0,1018.35,2021-02-02
3,https://moneywall.io/admin/credit_agreements/8150,0,MW333399856,,,,,2021-01,4.0,4832.10,4785.0,9617.10,7081.0,4785.0,1162.14,2021-02-02
4,https://moneywall.io/admin/credit_agreements/8068,0,MW333396121,,,,,2021-02,2.0,2289.16,0.0,2289.16,3374.8,2281.0,587.86,2021-02-02
6,https://moneywall.io/admin/credit_agreements/8154,0,MW333399885,,,,,2021-01,4.0,4077.61,4038.0,8115.61,5974.0,4038.0,980.45,2021-02-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2169,https://moneywall.io/admin/credit_agreements/848,1,MW333339982,,,,,2020-07,881.0,1297.00,0.0,1297.00,11631.0,6281.0,931.00,2021-02-02
2186,https://moneywall.io/admin/credit_agreements/1360,1,MW333343659,,,,,2020-09,683.0,697.00,0.0,697.00,3592.0,1932.0,272.00,2021-02-02
2187,https://moneywall.io/admin/credit_agreements/4758,1,MW333364748,,,,,2020-03,339.0,342.81,0.0,342.81,2003.4,1354.0,68.23,2021-02-02
2189,https://moneywall.io/admin/credit_agreements/1505,1,MW333344049,,,,,2020-03,640.0,568.00,0.0,568.00,12469.0,6708.0,947.00,2021-02-02
